In [6]:
%pip install matplotlib
%pip install pandas
%pip install numpy
%pip install Bio

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\adm\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\adm\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\adm\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\adm\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [7]:
import pandas as pd
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [44]:
os.getcwd()

'c:\\Users\\gchir\\Desktop\\bio\\gut-microbiota-diet'

In [45]:
data = pd.read_csv('sample_information_American_gut_microbiome.csv.txt', sep="\t", dtype=object)
print('Numero di campioni: ', data.shape[0], '\nInformazioni: ', data.shape[1]-1)

Numero di campioni:  29960 
Informazioni:  660


In [46]:
data.head()

,sample_name,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_cat,age_corrected,age_years,alcohol_consumption,alcohol_frequency,...,vioscreen_zinc,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_cat,weight_change,weight_kg,weight_units,whole_eggs,whole_grain_frequency
0,10317.000001000,Not provided,false,false,"Diagnosed by a medical professional (doctor, p...",60s,64.0,64,true,Daily,...,Not provided,Never,Regularly (3-5 times/week),Not provided,NaN,Remained stable,52,kilograms,Never,Occasionally (1-2 times/week)
1,10317.000001001,Not provided,false,false,Not provided,50s,53.0,53.0,true,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,110.0,kilograms,Not provided,Not provided
2,10317.000001002,Not provided,false,false,Not provided,50s,53.0,53.0,true,Regularly (3-5 times/week),...,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,56.0,kilograms,Not provided,Not provided
3,10317.000001004,Not provided,false,false,Not provided,40s,44.0,44.0,true,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,86.0,kilograms,Not provided,Not provided
4,10317.000001008,Not provided,false,false,Not provided,60s,66.0,66.0,false,Never,...,Not provided,Not provided,Not provided,Not provided,NaN,Increased more than 10 pounds,74.0,kilograms,Not provided,Not provided


In [47]:
data[data=='true']='Yes'

In [48]:
data[data=='false']='No'
data[data=='I do not have this condition']='No'

In [49]:
data['lactose'].value_counts()

lactose
No                 20422
Yes                 3802
Not applicable      3598
Not provided         773
LabControl test      681
Unspecified          594
not applicable        10
not collected          3
Name: count, dtype: int64

In [50]:
list(data.columns)

['sample_name',
 'acid_reflux',
 'acne_medication',
 'acne_medication_otc',
 'add_adhd',
 'age_cat',
 'age_corrected',
 'age_years',
 'alcohol_consumption',
 'alcohol_frequency',
 'alcohol_types',
 'alcohol_types_beercider',
 'alcohol_types_red_wine',
 'alcohol_types_sour_beers',
 'alcohol_types_spiritshard_alcohol',
 'alcohol_types_unspecified',
 'alcohol_types_white_wine',
 'allergic_to',
 'allergic_to_i_have_no_food_allergies_that_i_know_of',
 'allergic_to_other',
 'allergic_to_peanuts',
 'allergic_to_shellfish',
 'allergic_to_tree_nuts',
 'allergic_to_unspecified',
 'altitude',
 'alzheimers',
 'animal_age',
 'animal_free_text',
 'animal_gender',
 'animal_origin',
 'animal_type',
 'anonymized_name',
 'antibiotic_history',
 'appendix_removed',
 'artificial_sweeteners',
 'asd',
 'assigned_from_geo',
 'autoimmune',
 'birth_year',
 'bmi',
 'bmi_cat',
 'bmi_corrected',
 'body_habitat',
 'body_product',
 'body_site',
 'bowel_movement_frequency',
 'bowel_movement_quality',
 'breastmilk_for

In [51]:
data.specialized_diet_exclude_refined_sugars.value_counts()

specialized_diet_exclude_refined_sugars
Not provided       13188
No                 10029
Not applicable      3598
Yes                 2101
LabControl test      681
Unspecified          203
not applicable        10
not collected          3
Name: count, dtype: int64

In [52]:
data.sibo.value_counts()

sibo
No                                                                   18152
Not provided                                                          5580
Not applicable                                                        3598
LabControl test                                                        681
Unspecified                                                            660
Self-diagnosed                                                         494
Diagnosed by a medical professional (doctor, physician assistant)      445
Diagnosed by an alternative medicine practitioner                      260
not applicable                                                          10
not collected                                                            3
Name: count, dtype: int64

In [53]:
data = data.loc[data.body_site == "UBERON:feces",:]

In [54]:
cond_no_disease=(((data['acid_reflux']=='No') | (data['acid_reflux']=='Not provided')) &
(data['alcohol_frequency']!='Daily') &
(data['allergic_to_peanuts']== 'No') &
(data['allergic_to_shellfish']== 'No') &
(data['allergic_to_tree_nuts']=='No') &
(data['allergic_to_other']=='No') &
((data['autoimmune']=='No') | (data['autoimmune']=='Not provided')) &
((data['cancer']=='No') | (data['cancer']=='Not provided')) &
((data['cdiff']=='No') | (data['cdiff']=='Not provided')) &
((data['diabetes']=='No') | (data['diabetes']=='Not provided')) &
((data['gluten']=='No') | (data['gluten']=='Not provided')) &
((data['ibd']=='No') | (data['ibd']=='Not provided')) &
((data['ibs']=='No') | (data['ibs']=='Not provided')) &
((data['kidney_disease']=='No') | (data['kidney_disease']=='Not provided')) &
((data['lactose']=='No') | (data['lactose']=='Not provided')) &
((data['liver_disease']=='No') | (data['liver_disease']=='Not provided')) &
((data['mental_illness_type_anorexia_nervosa']=='No') | (data['mental_illness_type_anorexia_nervosa']=='Not provided')) &
((data['mental_illness_type_bulimia_nervosa']=='No') | (data['mental_illness_type_bulimia_nervosa']=='Not provided')) &
((data['sibo']=='No') | (data['sibo']=='Not provided')))

In [55]:
data_no_disease = data.loc[cond_no_disease, :]

In [56]:
data_no_disease

,sample_name,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_cat,age_corrected,age_years,alcohol_consumption,alcohol_frequency,...,vioscreen_zinc,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_cat,weight_change,weight_kg,weight_units,whole_eggs,whole_grain_frequency
1,10317.000001001,Not provided,No,No,Not provided,50s,53.0,53.0,Yes,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,110.0,kilograms,Not provided,Not provided
2,10317.000001002,Not provided,No,No,Not provided,50s,53.0,53.0,Yes,Regularly (3-5 times/week),...,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,56.0,kilograms,Not provided,Not provided
3,10317.000001004,Not provided,No,No,Not provided,40s,44.0,44.0,Yes,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,86.0,kilograms,Not provided,Not provided
4,10317.000001008,Not provided,No,No,Not provided,60s,66.0,66.0,No,Never,...,Not provided,Not provided,Not provided,Not provided,NaN,Increased more than 10 pounds,74.0,kilograms,Not provided,Not provided
6,10317.000001018,Not provided,No,Yes,Not provided,20s,28.0,28.0,Yes,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,77.0,kilograms,Not provided,Not provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26302,10317.000115230,No,No,No,No,20s,27,27,Yes,Regularly (3-5 times/week),...,LabControl test,Rarely (a few times/month),Occasionally (1-2 times/week),Regularly (3-5 times/week),NaN,Decreased more than 10 pounds,96,kilograms,Daily,Occasionally (1-2 times/week)
26333,10317.000115677,No,No,No,No,Unspecified,Unspecified,5,No,Never,...,LabControl test,Never,Daily,Never,NaN,Remained stable,Unspecified,kilograms,Never,Never
26334,10317.000115686,No,No,No,No,Unspecified,Unspecified,1,No,Never,...,LabControl test,Never,Daily,Never,NaN,Remained stable,12,kilograms,Occasionally (1-2 times/week),Regularly (3-5 times/week)
26345,10317.000116371,No,No,No,No,60s,64,64,Yes,Rarely (a few times/month),...,LabControl test,Never,Never,Rarely (a few times/month),NaN,Remained stable,99,kilograms,Regularly (3-5 times/week),Occasionally (1-2 times/week)


In [57]:
column_filt = ["sample_name",
               "age_years",
               "age_cat",
               "sex",
               "specialized_diet_exclude_dairy",
               "specialized_diet_exclude_refined_sugars",
               "body_site",
               "specialized_diet_i_do_not_eat_a_specialized_diet",
               "specialized_diet_paleodiet_or_primal_diet"]

In [58]:
data_f = data_no_disease.loc[:,column_filt]

In [59]:
data_f.loc[data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "false","specialized_diet_i_do_not_eat_a_specialized_diet"] = "No"
data_f.loc[data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "true","specialized_diet_i_do_not_eat_a_specialized_diet"] = "Yes"

data_f.loc[data_f.specialized_diet_exclude_dairy == "false","specialized_diet_exclude_dairy"] = "No"
data_f.loc[data_f.specialized_diet_exclude_dairy == "true","specialized_diet_exclude_dairy"] = "Yes"

data_f.loc[data_f.specialized_diet_exclude_refined_sugars == "false","specialized_diet_exclude_refined_sugars"] = "No"
data_f.loc[data_f.specialized_diet_exclude_refined_sugars == "true","specialized_diet_exclude_refined_sugars"] = "Yes"

In [60]:
data_diet = data_f.loc[(data_f.specialized_diet_exclude_dairy == "No") |
                        (data_f.specialized_diet_exclude_dairy == "Yes") |
                        (data_f.specialized_diet_exclude_refined_sugars == "No") |
                        (data_f.specialized_diet_exclude_refined_sugars == "Yes")|
                        (data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "No") |
                        (data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "Yes"),:]

In [61]:
data_diet

,sample_name,age_years,age_cat,sex,specialized_diet_exclude_dairy,specialized_diet_exclude_refined_sugars,body_site,specialized_diet_i_do_not_eat_a_specialized_diet,specialized_diet_paleodiet_or_primal_diet
74,10317.000001166,47,40s,female,No,No,UBERON:feces,Yes,No
104,10317.000001226,67,60s,male,No,No,UBERON:feces,No,No
141,10317.000001300,50,50s,female,No,No,UBERON:feces,No,No
555,10317.000002224,36,30s,male,No,No,UBERON:feces,No,No
887,10317.000003009,63,60s,male,No,No,UBERON:feces,No,No
...,...,...,...,...,...,...,...,...,...
26302,10317.000115230,27,20s,male,No,Yes,UBERON:feces,Yes,No
26333,10317.000115677,5,Unspecified,male,No,No,UBERON:feces,Yes,No
26334,10317.000115686,1,Unspecified,female,No,No,UBERON:feces,Yes,No
26345,10317.000116371,64,60s,male,No,No,UBERON:feces,Yes,No


In [62]:
pd.crosstab(data_diet.specialized_diet_paleodiet_or_primal_diet	,
            data_diet.specialized_diet_exclude_refined_sugars, 
            values=data_diet.sample_name,
            aggfunc=len, 
            margins = True)

specialized_diet_exclude_refined_sugars,No,Yes,All
specialized_diet_paleodiet_or_primal_diet,,,
No,2958,287,3245
Yes,16,19,35
All,2974,306,3280


In [63]:
pd.crosstab(data_diet.sex,
            data_diet.specialized_diet_exclude_dairy, 
            values=data_diet.sample_name,
            aggfunc=len, 
            margins = True)

specialized_diet_exclude_dairy,No,Yes,All
sex,,,
Not provided,46.0,NaN,46
female,1524.0,65.0,1589
male,1572.0,66.0,1638
other,2.0,NaN,2
unspecified,5.0,NaN,5
All,3149.0,131.0,3280


In [64]:
pd.crosstab(data_diet.age_cat,
            data_diet.specialized_diet_exclude_dairy, 
            values=data_diet.sample_name,
            aggfunc=len, 
            margins = True)

specialized_diet_exclude_dairy,No,Yes,All
age_cat,,,
20s,400,19,419
30s,628,20,648
40s,546,23,569
50s,527,25,552
60s,378,13,391
70+,130,5,135
Not provided,102,4,106
Unspecified,124,14,138
child,190,4,194


In [65]:
list(data_diet.age_cat.unique())

['40s',
 '60s',
 '50s',
 '30s',
 'Not provided',
 '20s',
 'teen',
 'Unspecified',
 'child',
 '70+']

In [66]:
age_categ = ['20s', '30s', '40s', '50s']

In [67]:
data_diet_f = data_diet[((data_diet['age_cat'] != 'Unspecified') &
                      (data_diet['age_years'] != 'Not provided') &
                      (data_diet['age_cat'] != 'child') &
                      (data_diet['age_cat'] != 'teen'))  &
                      ((data_diet.sex == 'female') |
                        (data_diet.sex == 'male')) &
                        ((data_diet['age_cat'] != '60s') | (data_diet['age_cat'] != '70s'))]

In [68]:
data_diet_f.describe

<bound method NDFrame.describe of            sample_name age_years age_cat     sex   
74     10317.000001166        47     40s  female  \
104    10317.000001226        67     60s    male   
141    10317.000001300        50     50s  female   
555    10317.000002224        36     30s    male   
887    10317.000003009        63     60s    male   
...                ...       ...     ...     ...   
26291  10317.000115216        63     60s  female   
26293  10317.000115219        62     60s  female   
26302  10317.000115230        27     20s    male   
26345  10317.000116371        64     60s    male   
26349  10317.000116379        56     50s    male   

      specialized_diet_exclude_dairy specialized_diet_exclude_refined_sugars   
74                                No                                      No  \
104                               No                                      No   
141                               No                                      No   
555                  

In [69]:
data_diet_f.head(5)

,sample_name,age_years,age_cat,sex,specialized_diet_exclude_dairy,specialized_diet_exclude_refined_sugars,body_site,specialized_diet_i_do_not_eat_a_specialized_diet,specialized_diet_paleodiet_or_primal_diet
74,10317.000001166,47,40s,female,No,No,UBERON:feces,Yes,No
104,10317.000001226,67,60s,male,No,No,UBERON:feces,No,No
141,10317.000001300,50,50s,female,No,No,UBERON:feces,No,No
555,10317.000002224,36,30s,male,No,No,UBERON:feces,No,No
887,10317.000003009,63,60s,male,No,No,UBERON:feces,No,No


In [70]:
data_diet_f.specialized_diet_exclude_refined_sugars.value_counts()

specialized_diet_exclude_refined_sugars
No     2464
Yes     279
Name: count, dtype: int64

In [71]:
data_diet_f.age_years = data_diet_f.age_years.astype(int)

C:\Users\gchir\AppData\Local\Temp\ipykernel_11904\59878515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_diet_f.age_years = data_diet_f.age_years.astype(int)


In [72]:
np.sum((data_diet_f.specialized_diet_exclude_dairy == "No") & 
                            (data_diet_f.specialized_diet_exclude_refined_sugars == "Yes") &
                            (data_diet_f.specialized_diet_i_do_not_eat_a_specialized_diet == "No")) 

214

In [73]:
refined_sugar_sample = data_diet_f.loc[data_diet_f['specialized_diet_exclude_refined_sugars'] == 'Yes']

In [74]:
refined_sugar_sample

,sample_name,age_years,age_cat,sex,specialized_diet_exclude_dairy,specialized_diet_exclude_refined_sugars,body_site,specialized_diet_i_do_not_eat_a_specialized_diet,specialized_diet_paleodiet_or_primal_diet
6610,10317.000023139,69,60s,female,No,Yes,UBERON:feces,No,No
6809,10317.000023608,59,50s,male,No,Yes,UBERON:feces,No,No
8522,10317.000030449,27,20s,male,No,Yes,UBERON:feces,No,Yes
9440,10317.000033280,60,60s,female,No,Yes,UBERON:feces,No,No
9804,10317.000035367,41,40s,female,No,Yes,UBERON:feces,No,Yes
...,...,...,...,...,...,...,...,...,...
25831,10317.000113033,38,30s,male,No,Yes,UBERON:feces,Yes,No
25834,10317.000113037,49,40s,female,No,Yes,UBERON:feces,No,No
26079,10317.000114439,55,50s,male,No,Yes,UBERON:feces,No,No
26116,10317.000114499,22,20s,male,No,Yes,UBERON:feces,No,No


FastQ research:

In [75]:
from Bio import Entrez
Entrez.email = 'grusso@campus.unimib.it'

import xml.etree.ElementTree as ET
import os

In [92]:
def from_samplenames_to_fastqa(sample_names_column, download = False):
    
    # Inizializzo le liste vuote che poi riempirò durante il ciclo
    sample_names = []
    biosample_ids = []
    sra_ids = []
    sra_names = []
    sra_search_res = []
    run_ids = []    
    names_without_biosamples = []
    names_without_runs = []
    
    tot = len(sample_names_column)
    for i in range(len(sample_names_column)):
        
        sample = sample_names_column.iloc[i]
        print('\n', i)
        print('Sample name: ', sample)

        # Da sample name a biosample id:
        handleSce = Entrez.esearch(db='biosample', term=sample)
        recordSce = Entrez.read(handleSce)
        print('--Numero di BiosampleIDs trovati: ', len(recordSce['IdList']))

        if len(recordSce['IdList']) == 0:
            names_without_biosamples.append(sample)
            
        else:
            for b in recordSce['IdList']:
            
                print('----BioSampleID: ', b)

                # Da biosample id a SRA id
                try:
                    handleID = Entrez.efetch(db='biosample', id=b, retmode='xml')
                    root = ET.fromstring(handleID.read())
                    identifier = root.findall('.//BioSample//Ids//Id')
                except:
                    print('XML not found')
                    continue
                

                for i in identifier:
                    if i.attrib['db'] == 'SRA':
                        sraId = i.text

                        print('------SRA id: ', sraId)

                        # Da SRAid a RUNid
                        handleSra = Entrez.esearch(db='sra', term=sraId,retmode="xml")
                        resultsSra = Entrez.read(handleSra)["IdList"]



                        print('--------SRA search results: ', resultsSra)

                        for s in resultsSra:

                            handlesngSraId = Entrez.efetch(db='sra', id=s)
                            root = ET.fromstring(handlesngSraId.read())   
                            identifier = root.find('.//EXPERIMENT_PACKAGE//RUN_SET//RUN')
                            instrument = root.find('.//EXPERIMENT_PACKAGE//EXPERIMENT//PLATFORM//ILLUMINA//INSTRUMENT_MODEL')
                            
                            
                            print(instrument.text)
                            if (identifier != None) & (instrument.text == "Illumina MiSeq"):
                                runId = identifier.attrib['accession']

                                print('----------Run ID: ', runId)
                                
                                sample_names.append(sample)
                                biosample_ids.append(b)
                                sra_ids.append(sraId)
                                sra_search_res.append(s)
                                run_ids.append(runId)

                                # Download FASQ e FASTA dei RUNid
                                if download:

                                    print('------------Downloading FASTQ...')
                                    command2 = 'sratoolkit.3.0.1-win64\\bin\\fastq-dump.exe --outdir fastq --readids ' + runId
                                    os.system(command2)
                               
                            else:
                                names_without_runs.append(sample)

    dict = {'sample_names': sample_names, 
            'biosample_ids': biosample_ids, 
            'sra_ids': sra_ids,
            'sra_search_res': sra_search_res,
            'run_ids': run_ids,
            'names_without_biosamples': names_without_biosamples,
            'names_without_runs': names_without_runs}  

    return dict    

In [94]:
tmp = from_samplenames_to_fastqa(refined_sugar_sample.sample_name, download = True)


 0
Sample name:  10317.000023139
--Numero di BiosampleIDs trovati:  1
----BioSampleID:  7353729
------SRA id:  ERS1821637
--------SRA search results:  ['4279800']
Illumina MiSeq
----------Run ID:  ERR2032495
------------Downloading FASTQ...

 1
Sample name:  10317.000023608
--Numero di BiosampleIDs trovati:  1
----BioSampleID:  8568870
------SRA id:  ERS2203044
--------SRA search results:  ['5125216']
Illumina MiSeq
----------Run ID:  ERR2303818
------------Downloading FASTQ...

 2
Sample name:  10317.000030449
--Numero di BiosampleIDs trovati:  1
----BioSampleID:  9541103
------SRA id:  ERS2483074
--------SRA search results:  ['5840380']
Illumina MiSeq
----------Run ID:  ERR2579642
------------Downloading FASTQ...

 3
Sample name:  10317.000033280
--Numero di BiosampleIDs trovati:  1
----BioSampleID:  8577201
------SRA id:  ERS2213544
--------SRA search results:  ['5144891']
Illumina MiSeq
----------Run ID:  ERR2313963
------------Downloading FASTQ...

 4
Sample name:  10317.00003536

# FASTQ - Quality check

In [1]:
from Bio import SeqIO
from os import listdir
from os.path import isfile, join

In [3]:
#%matplotlib inline
#plt.rcParams['figure.figsize'] = [5,3]

# Funzione per filtrare solo le letture con una qualità maggiore di X
def filter(file, x):
    for rec in SeqIO.parse('fastq/' + file, "fastq"):
        if min(rec.letter_annotations["phred_quality"]) >= x:
            yield rec

def apply_filter(filelist, quality_min_required, image = False):
    
    if not os.path.exists('fastq/goodReads_' + str(quality_min_required)):
        os.mkdir('fastq/goodReads_' + str(quality_min_required))
    
    qualities_min_min = {}
    qualities_min_max = {}
    qualities_min_avg = {}
    starting_reads = {}
    good_reads ={}
    
    tot = len(filelist)
    pos = 0
    
    for f in filelist:
        pos +=1
        
        print('\n', pos, '/', tot)       
        print('File name: ', f)
        print('--File parsing...')

        fileParsed = SeqIO.parse('fastq/' + f, "fastq")
        print(' ',fileParsed)
        print('--Reads:')
        count = 0

        # Conto le letture presenti nel file e la qualità minima di ognuna
        quality_min = []

        for rec in fileParsed:
            count += 1
            # print('----', rec)
            # print('quality min: ', min(rec.letter_annotations['phred_quality']))
            # print(rec.letter_annotations["phred_quality"])

            quality_min.append(min(rec.letter_annotations['phred_quality']))

        qualities_min_min[f] = min(quality_min)
        qualities_min_max[f] = max(quality_min)
        qualities_min_avg[f] = sum(quality_min)/len(quality_min)

        print("----Reads Totali: ", count)
        starting_reads[f] = count

        # Filtro solo le letture "buone"
        goodReads = filter(f, quality_min_required)
        count = SeqIO.write(goodReads, 'fastq/goodReads_' + str(quality_min_required) + '/good_' + str(quality_min_required) + '_' + f, "fastq") # "output/good_quality.fastq"

        print("----Reads Mantenute: ", count)
        good_reads[f] = count

        # Scrivo anche il fasta con le sole letture buone
        #SeqIO.convert ('rawData/fastq/goodReads/good_'+f, 'fastq', 
        #               'rawData/fasta/goodReads/good_'+f.replace("fastq", "fasta"), 'fasta')

        # Qualità minima delle letture del file:
        if image:
            plt.hist(quality_min, density=False)
            plt.suptitle(f.replace("fastq", ""))
            plt.savefig('images/qualityMin/' + f.replace("fastq", "png"))
            plt.show()# `density=False` would make counts
        
    goodness_statistics = dict([(k, [starting_reads[k], 
                             good_reads[k],
                             qualities_min_min[k],
                             qualities_min_max[k],
                             qualities_min_avg[k]]) for k in filelist])

    goodness_statistics = pd.DataFrame(goodness_statistics).T
    goodness_statistics.reset_index(level=0, inplace=True)
    goodness_statistics.columns = ['fastq_file', 
                                   'starting_reads', 
                                   'good_reads', 
                                   'quality_min_min', 
                                   'quality_min_max', 
                                   'quality_min_avg']
    
    goodness_statistics['perc_good'] = goodness_statistics.good_reads / goodness_statistics.starting_reads
    
    return goodness_statistics

In [8]:
fastqfiles = [f for f in listdir('fastq/') if isfile(join('fastq/', f))]
len(fastqfiles)
fastqfiles_series = pd.Series(fastqfiles)
len(fastqfiles_series)

268

In [9]:
goodness_statistics_20 = apply_filter(fastqfiles_series, quality_min_required = 20,  image = False)
#goodness_statistics_20.to_csv('datiIntermedi/goodness_statistics_20.csv', index = False)


 1 / 268
File name:  ERR1842797.fastq
--File parsing...
--Reads:
----Reads Totali:  28329
----Reads Mantenute:  23

 2 / 268
File name:  ERR1842808.fastq
--File parsing...
--Reads:
----Reads Totali:  16660
----Reads Mantenute:  98

 3 / 268
File name:  ERR1842816.fastq
--File parsing...
--Reads:
----Reads Totali:  23548
----Reads Mantenute:  124

 4 / 268
File name:  ERR1842877.fastq
--File parsing...
--Reads:
----Reads Totali:  17826
----Reads Mantenute:  24

 5 / 268
File name:  ERR2032284.fastq
--File parsing...
--Reads:
----Reads Totali:  46561
----Reads Mantenute:  9318

 6 / 268
File name:  ERR2032285.fastq
--File parsing...
--Reads:
----Reads Totali:  28402
----Reads Mantenute:  5994

 7 / 268
File name:  ERR2032286.fastq
--File parsing...
--Reads:
----Reads Totali:  22507
----Reads Mantenute:  5125

 8 / 268
File name:  ERR2032297.fastq
--File parsing...
--Reads:
----Reads Totali:  26714
----Reads Mantenute:  6215

 9 / 268
File name:  ERR2032371.fastq
--File parsing...
--Read

In [12]:
goodness_statistics_20.head(10)

,fastq_file,starting_reads,good_reads,quality_min_min,quality_min_max,quality_min_avg,perc_good
0,ERR1842797.fastq,28329.0,23.0,12.0,33.0,12.434996,0.000812
1,ERR1842808.fastq,16660.0,98.0,12.0,34.0,12.550300,0.005882
2,ERR1842816.fastq,23548.0,124.0,12.0,33.0,12.821811,0.005266
3,ERR1842877.fastq,17826.0,24.0,12.0,33.0,12.386290,0.001346
4,ERR2032284.fastq,46561.0,9318.0,12.0,34.0,16.891884,0.200125
5,ERR2032285.fastq,28402.0,5994.0,12.0,34.0,17.279628,0.211041
6,ERR2032286.fastq,22507.0,5125.0,12.0,34.0,17.623939,0.227707
7,ERR2032297.fastq,26714.0,6215.0,12.0,34.0,17.679756,0.232650
8,ERR2032371.fastq,27191.0,5614.0,12.0,34.0,17.047663,0.206465
9,ERR2032372.fastq,23010.0,4714.0,12.0,34.0,17.167753,0.204867


In [11]:
goodness_statistics_20.to_csv('fastq_goodness.csv')

In [13]:
np.mean(goodness_statistics_20['quality_min_avg'])

19.490504048585244